# Basic Models

In [1]:
from pydantic import BaseModel

In [2]:
class User(BaseModel):
    id: int
    name = 'Jane Doe'

In [11]:
user = User(id=123)
user2 = User(id=234, name="Jack")

In [5]:
assert user.id == 123

In [7]:
assert user.__fields_set__ == {'id'}

In [12]:
# 在 `user` 初始化时，`name` 字段未提供值，使用的是默认值
user.__fields_set__

{'id'}

In [13]:
user2.__fields_set__

{'id', 'name'}

In [15]:
dict(user), user.dict()

({'id': 123, 'name': 'Jane Doe'}, {'id': 123, 'name': 'Jane Doe'})

# Recursive Models

In [16]:
from typing import List
from pydantic import BaseModel

In [17]:
class Foo(BaseModel):
    count: int
    size: float = None


class Bar(BaseModel):
    apple = 'x'
    banana = 'y'


class Spam(BaseModel):
    foo: Foo
    bars: List[Bar]

In [18]:
m = Spam(foo={'count': 4}, bars=[{'apple': 'x1'}, {'apple': 'x2'}])
m

Spam(foo=Foo(count=4, size=None), bars=[Bar(apple='x1', banana='y'), Bar(apple='x2', banana='y')])

In [19]:
m.dict()

{'foo': {'count': 4, 'size': None},
 'bars': [{'apple': 'x1', 'banana': 'y'}, {'apple': 'x2', 'banana': 'y'}]}

# ORM Mode

In [20]:
from typing import List
from sqlalchemy import Column, Integer, String
from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.ext.declarative import declarative_base
from pydantic import BaseModel, constr

In [23]:
Base = declarative_base()


class CompanyOrm(Base):
    __tablename__ = 'companies'
    id = Column(Integer, primary_key=True, nullable=False)
    public_key = Column(String(20), index=True, nullable=False, unique=True)
    name = Column(String(63), unique=True)
    domains = Column(ARRAY(String(255)))


class CompanyModel(BaseModel):
    id: int
    public_key: constr(max_length=20)
    name: constr(max_length=63)
    domains: List[constr(max_length=255)]

    class Config:
        orm_mode = True

In [25]:
co_orm = CompanyOrm(
    id=123,
    public_key='foobar',
    name='Testing',
    domains=['example.com', 'foobar.com'],
)
co_model = CompanyModel.from_orm(co_orm)
co_model

CompanyModel(id=123, public_key='foobar', name='Testing', domains=['example.com', 'foobar.com'])

## Reserved Names

In [26]:
from typing import Dict

from pydantic import BaseModel, Field
from sqlalchemy import Column, Integer, JSON
from sqlalchemy.ext.declarative import declarative_base

In [33]:
class MyModel(BaseModel):
    metadata: Dict[str, str] = Field(alias='metadata_')

    class Config:
        orm_mode = True

NameError: name 'Dict' is not defined

In [ ]:
Base = declarative_base()


class SQLModel(Base):
    __tablename__ = 'my_table'
    id = Column('id', Integer, primary_key=True)
    # 'metadata' is reserved by SQLAlchemy, hence the '_'
    metadata_ = Column('metadata', JSON)

In [34]:
sql_model = SQLModel(metadata_={'key': 'val'}, id=1)
pydantic_model = MyModel.from_orm(sql_model)
pydantic_model.dict()

{'metadata': {'key': 'val'}}

In [35]:
pydantic_model.dict(by_alias=True)

{'metadata_': {'key': 'val'}}

## Recursive ORM models

In [32]:
from typing import List
from pydantic import BaseModel

In [37]:
class PetCls:
    def __init__(self, *, name: str, species: str):
        self.name = name
        self.species = species


class PersonCls:
    def __init__(self, *, name: str, age: float = None, pets: List[PetCls]):
        self.name = name
        self.age = age
        self.pets = pets


class Pet(BaseModel):
    name: str
    species: str

    class Config:
        orm_mode = True


class Person(BaseModel):
    name: str
    age: float = None
    pets: List[Pet]

    class Config:
        orm_mode = True

In [38]:
bones = PetCls(name='Bones', species='dog')
orion = PetCls(name='Orion', species='cat')
anna = PersonCls(name='Anna', age=20, pets=[bones, orion])
anna_model = Person.from_orm(anna)
anna_model

Person(name='Anna', age=20.0, pets=[Pet(name='Bones', species='dog'), Pet(name='Orion', species='cat')])

## Data Binding

In [39]:
from pydantic import BaseModel
from typing import Any, Optional
from pydantic.utils import GetterDict
from xml.etree.ElementTree import fromstring

In [40]:
xml_string = """
<User Id="2138">
    <FirstName />
    <LoggedIn Value="true" />
</User>
"""

In [41]:
class UserGetter(GetterDict):
    def get(self, key: str, default: Any) -> Any:

        # element attributes
        if key in {'Id', 'Status'}:
            return self._obj.attrib.get(key, default)

        # element children
        else:
            try:
                return self._obj.find(key).attrib['Value']
            except (AttributeError, KeyError):
                return default


class User(BaseModel):
    Id: int
    Status: Optional[str]
    FirstName: Optional[str]
    LastName: Optional[str]
    LoggedIn: bool

    class Config:
        orm_mode = True
        getter_dict = UserGetter

In [42]:
user = User.from_orm(fromstring(xml_string))
user.dict()

{'Id': 2138,
 'Status': None,
 'FirstName': None,
 'LastName': None,
 'LoggedIn': True}

# Error handling

In [44]:
from typing import List
from pydantic import BaseModel, ValidationError, conint

In [45]:
class Location(BaseModel):
    lat = 0.1
    lng = 10.1


class Model(BaseModel):
    is_required: float
    gt_int: conint(gt=42)
    list_of_ints: List[int] = None
    a_float: float = None
    recursive_model: Location = None

In [46]:
data = dict(
    list_of_ints=['1', 2, 'bad'],
    a_float='not a float',
    recursive_model={'lat': 4.2, 'lng': 'New York'},
    gt_int=21,
)

try:
    Model(**data)
except ValidationError as e:
    print(e)

5 validation errors for Model
is_required
  field required (type=value_error.missing)
gt_int
  ensure this value is greater than 42 (type=value_error.number.not_gt; limit_value=42)
list_of_ints -> 2
  value is not a valid integer (type=type_error.integer)
a_float
  value is not a valid float (type=type_error.float)
recursive_model -> lng
  value is not a valid float (type=type_error.float)


In [47]:
try:
    Model(**data)
except ValidationError as e:
    print(e.json())

[
  {
    "loc": [
      "is_required"
    ],
    "msg": "field required",
    "type": "value_error.missing"
  },
  {
    "loc": [
      "gt_int"
    ],
    "msg": "ensure this value is greater than 42",
    "type": "value_error.number.not_gt",
    "ctx": {
      "limit_value": 42
    }
  },
  {
    "loc": [
      "list_of_ints",
      2
    ],
    "msg": "value is not a valid integer",
    "type": "type_error.integer"
  },
  {
    "loc": [
      "a_float"
    ],
    "msg": "value is not a valid float",
    "type": "type_error.float"
  },
  {
    "loc": [
      "recursive_model",
      "lng"
    ],
    "msg": "value is not a valid float",
    "type": "type_error.float"
  }
]


## Custom Error

In [48]:
from pydantic import BaseModel, ValidationError, validator

In [49]:
class Model(BaseModel):
    foo: str

    @validator('foo')
    def value_must_equal_bar(cls, v):
        if v != 'bar':
            raise ValueError('value must be "bar"')

        return v

In [50]:
try:
    Model(foo='ber')
except ValidationError as e:
    print(e.errors())

[{'loc': ('foo',), 'msg': 'value must be "bar"', 'type': 'value_error'}]


In [51]:
from pydantic import BaseModel, PydanticValueError, ValidationError, validator

In [52]:
class NotABarError(PydanticValueError):
    code = 'not_a_bar'
    msg_template = 'value is not "bar", got "{wrong_value}"'


class Model(BaseModel):
    foo: str

    @validator('foo')
    def value_must_equal_bar(cls, v):
        if v != 'bar':
            raise NotABarError(wrong_value=v)
        return v

In [53]:
try:
    Model(foo='ber')
except ValidationError as e:
    print(e)

1 validation error for Model
foo
  value is not "bar", got "ber" (type=value_error.not_a_bar; wrong_value=ber)


# Helper Functions

In [55]:
import pickle
from datetime import datetime
from pathlib import Path
from pydantic import BaseModel, ValidationError

In [56]:
class User(BaseModel):
    id: int
    name = 'John Doe'
    signup_ts: datetime = None

In [57]:
m = User.parse_obj({'id': 123, 'name': 'James'})
m

User(id=123, signup_ts=None, name='James')

In [58]:
try:
    User.parse_obj(['not', 'a', 'dict'])
except ValidationError as e:
    print(e)

1 validation error for User
__root__
  User expected dict not list (type=type_error)


In [59]:
m = User.parse_raw('{"id": 123, "name": "James"}')
print(m)

id=123 signup_ts=None name='James'


In [60]:
pickle_data = pickle.dumps({
    'id': 123,
    'name': 'James',
    'signup_ts': datetime(2017, 7, 14)
})

m = User.parse_raw(
    pickle_data, content_type='application/pickle', allow_pickle=True
)
m

User(id=123, signup_ts=datetime.datetime(2017, 7, 14, 0, 0), name='James')

## Creating models without validation

In [61]:
from pydantic import BaseModel

In [62]:
class User(BaseModel):
    id: int
    age: int
    name: str = 'John Doe'

In [63]:
original_user = User(id=123, age=32)
user_data = original_user.dict()
user_data

{'id': 123, 'age': 32, 'name': 'John Doe'}

In [64]:
fields_set = original_user.__fields_set__
fields_set

{'age', 'id'}

In [65]:
new_user = User.construct(_fields_set=fields_set, **user_data)
repr(new_user)

"User(id=123, age=32, name='John Doe')"

In [66]:
new_user.__fields_set__

{'age', 'id'}

In [67]:
bad_user = User.construct(id='dog')
repr(bad_user)

"User(id='dog', name='John Doe')"

# Generic Models

In [89]:
from typing import Generic, TypeVar, Optional, List
from pydantic import BaseModel, validator, ValidationError
from pydantic.generics import GenericModel

DataT = TypeVar('DataT')


class Error(BaseModel):
    code: int
    message: str


class DataModel(BaseModel):
    numbers: List[int]
    people: List[str]


class Response(GenericModel, Generic[DataT]):
    data: Optional[DataT]
    error: Optional[Error]

    @validator('error', always=True)
    def check_consistency(cls, v, values):
        if v is not None and values['data'] is not None:
            raise ValueError('must not provide both data and error')

        if v is None and values.get('data') is None:
            raise ValueError('must provide data or error')

        return v

data = DataModel(numbers=[1, 2, 3], people=[])
error = Error(code=404, message='Not found')

In [71]:
Response[int](data=1)

Response[int](data=1, error=None)

In [72]:
Response[str](data='value')

Response[str](data='value', error=None)

In [73]:
Response[str](data='value').dict()

{'data': 'value', 'error': None}

In [74]:
Response[DataModel](data=data).dict()

{'data': {'numbers': [1, 2, 3], 'people': []}, 'error': None}

In [75]:
Response[DataModel](error=error).dict()

{'data': None, 'error': {'code': 404, 'message': 'Not found'}}


In [76]:
try:
    Response[int](data='value')
except ValidationError as e:
    print(e)

2 validation errors for Response[int]
data
  value is not a valid integer (type=type_error.integer)
error
  must provide data or error (type=value_error)


In [90]:
from typing import TypeVar, Generic
from pydantic.generics import GenericModel

TypeX = TypeVar('TypeX')
TypeY = TypeVar('TypeY')
TypeZ = TypeVar('TypeZ')


class BaseClass(GenericModel, Generic[TypeX, TypeY]):
    x: TypeX
    y: TypeY


class ChildClass(BaseClass[int, TypeY], Generic[TypeY, TypeZ]):
    z: TypeZ


ChildClass[str, int](x=1, y='y', z=3)

In [82]:
from typing import Generic, TypeVar, Type, Any, Tuple
from pydantic.generics import GenericModel

DataT = TypeVar('DataT')


class Response(GenericModel, Generic[DataT]):
    data: DataT

    @classmethod
    def __concrete_name__(cls: Type[Any], params: Tuple[Type[Any], ...]) -> str:
        return f'{params[0].__name__.title()}Response'

repr(Response[int](data=1))

In [88]:
repr(Response[str](data='a'))

"StrResponse(data='a')"

In [91]:
from typing import Generic, TypeVar
from pydantic import ValidationError
from pydantic.generics import GenericModel

T = TypeVar('T')


class InnerT(GenericModel, Generic[T]):
    inner: T


class OuterT(GenericModel, Generic[T]):
    outer: T
    nested: InnerT[T]


nested = InnerT[int](inner=1)
OuterT[int](outer=1, nested=nested)

OuterT[int](outer=1, nested=InnerT[int](inner=1))

In [92]:
try:
    nested = InnerT[str](inner='a')
    print(OuterT[int](outer='a', nested=nested))
except ValidationError as e:
    print(e)

2 validation errors for OuterT[int]
outer
  value is not a valid integer (type=type_error.integer)
nested -> inner
  value is not a valid integer (type=type_error.integer)


In [93]:
from typing import Generic, TypeVar
from pydantic import ValidationError
from pydantic.generics import GenericModel

AT = TypeVar('AT')
BT = TypeVar('BT')


class Model(GenericModel, Generic[AT, BT]):
    a: AT
    b: BT


Model(a='a', b='a')

Model(a='a', b='a')

In [94]:
IntT = TypeVar('IntT', bound=int)
typeVarModel = Model[int, IntT]
typeVarModel(a=1, b=1)

Model[int, IntT](a=1, b=1)

In [95]:
try:
    typeVarModel(a='a', b='a')
except ValidationError as exc:
    print(exc)

2 validation errors for Model[int, IntT]
a
  value is not a valid integer (type=type_error.integer)
b
  value is not a valid integer (type=type_error.integer)


# Dynamic Model Creation

In [96]:
from pydantic import BaseModel, create_model

DynamicFoobarModel = create_model('DynamicFoobarModel', foo=(str, ...), bar=123)


class StaticFoobarModel(BaseModel):
    foo: str
    bar: int = 123

In [97]:
from pydantic import BaseModel, create_model


class FooModel(BaseModel):
    foo: str
    bar: int = 123


BarModel = create_model(
    'BarModel',
    apple='russet',
    banana='yellow',
    __base__=FooModel,
)
BarModel

pydantic.main.BarModel

In [98]:
BarModel.__fields__.keys()

dict_keys(['foo', 'bar', 'apple', 'banana'])

In [99]:
from pydantic import create_model, ValidationError, validator


def username_alphanumeric(cls, v):
    """`isalnum` 用于验证字符串中至少有一个字符，并且所有字符都是字母或数字"""
    assert v.isalnum(), 'must be alphanumeric'
    return v


validators = {
    'username_validator':
    validator('username')(username_alphanumeric)
}

UserModel = create_model(
    'UserModel',
    username=(str, ...),
    __validators__=validators
)

user = UserModel(username='scolvin')
user

UserModel(username='scolvin')

In [100]:
try:
    UserModel(username='scolvi%n')
except ValidationError as e:
    print(e)

1 validation error for UserModel
username
  must be alphanumeric (type=assertion_error)


# Model Creation from `NamedTuple` or `TypedDict`

In [107]:
from typing_extensions import TypedDict
from pydantic import ValidationError, create_model_from_typeddict

In [108]:
class User(TypedDict):
    name: str
    id: int


class Config:
    extra = 'forbid'


UserM = create_model_from_typeddict(User, __config__=Config)
repr(UserM(name=123, id='3'))

"User(name='123', id=3)"

In [116]:
try:
    UserM(name=123, id='3', other='no')
except ValidationError as e:
    print(e)

1 validation error for User
other
  extra fields not permitted (type=value_error.extra)


```python
class Config:
    extra = 'forbid'
```

这个 `extra` 设置为 `forbid` 表示除了 `User` 类定义的 `id` 和 `name` 字段，其余字段都不被允许传入

# Custom Root Types

In [117]:
from typing import List
import json
from pydantic import BaseModel
from pydantic.schema import schema


class Pets(BaseModel):
    __root__: List[str]

In [118]:
Pets(__root__=['dog', 'cat'])

Pets(__root__=['dog', 'cat'])

In [119]:
Pets(__root__=['dog', 'cat']).json()

'["dog", "cat"]'

In [120]:
Pets.parse_obj(['dog', 'cat'])

Pets(__root__=['dog', 'cat'])

In [121]:
Pets.schema()

{'title': 'Pets', 'type': 'array', 'items': {'type': 'string'}}

In [123]:
pets_schema = schema([Pets])
print(json.dumps(pets_schema, indent=2))

{
  "definitions": {
    "Pets": {
      "title": "Pets",
      "type": "array",
      "items": {
        "type": "string"
      }
    }
  }
}


In [124]:
from typing import List, Dict
from pydantic import BaseModel, ValidationError


class Pets(BaseModel):
    __root__: List[str]


Pets.parse_obj(['dog', 'cat'])

Pets(__root__=['dog', 'cat'])

In [125]:
# 下面这种方式并不推荐使用
Pets.parse_obj({'__root__': ['dog', 'cat']})

Pets(__root__=['dog', 'cat'])

In [126]:
class PetsByName(BaseModel):
    __root__: Dict[str, str]


PetsByName.parse_obj({'Otis': 'dog', 'Milo': 'cat'})

PetsByName(__root__={'Otis': 'dog', 'Milo': 'cat'})

In [127]:
try:
    PetsByName.parse_obj({'__root__': {'Otis': 'dog', 'Milo': 'cat'}})
except ValidationError as e:
    print(e)

1 validation error for PetsByName
__root__ -> __root__
  str type expected (type=type_error.str)


In [128]:
from typing import List
from pydantic import BaseModel


class Pets(BaseModel):
    __root__: List[str]

    def __iter__(self):
        return iter(self.__root__)

    def __getitem__(self, item):
        return self.__root__[item]


pets = Pets.parse_obj(['dog', 'cat'])
print(pets[0])
print([pet for pet in pets])

dog
['dog', 'cat']


# Faux Immutability

In [129]:
from pydantic import BaseModel


class FooBarModel(BaseModel):
    a: str
    b: dict

    class Config:
        allow_mutation = False


foobar = FooBarModel(a='hello', b={'apple': 'pear'})

try:
    foobar.a = 'different'
except TypeError as e:
    print(e)

"FooBarModel" is immutable and does not support item assignment


# Abstract Base Classes

In [130]:
import abc
from pydantic import BaseModel


class FooBarModel(BaseModel, abc.ABC):
    a: str
    b: int

    @abc.abstractmethod
    def my_abstract_method(self):
        pass

# Field Ordering

In [131]:
from pydantic import BaseModel, ValidationError


class Model(BaseModel):
    a: int
    b = 2
    c: int = 1
    d = 0
    e: float


Model.__fields__.keys()

dict_keys(['a', 'c', 'e', 'b', 'd'])

In [132]:
m = Model(e=2, a=1)
m.dict()

{'a': 1, 'c': 1, 'e': 2.0, 'b': 2, 'd': 0}

In [134]:
try:
    Model(a='x', b='x', c='x', d='x', e='x')
except ValidationError as e:
    error_locations = [e['loc'] for e in e.errors()]
    print(error_locations)

[('a',), ('c',), ('e',), ('b',), ('d',)]


# Required fields

In [ ]:
from pydantic import BaseModel, Field


class Model(BaseModel):
    a: int
    b: int = ...
    c: int = Field(...)

# Required Optional fields

In [135]:
from typing import Optional
from pydantic import BaseModel, Field, ValidationError


class Model(BaseModel):
    a: Optional[int]
    b: Optional[int] = ...
    c: Optional[int] = Field(...)


Model(b=1, c=2)

Model(a=None, b=1, c=2)

In [136]:
try:
    Model(a=1, b=2)
except ValidationError as e:
    print(e)

1 validation error for Model
c
  field required (type=value_error.missing)


# Field with Dynamic Default Value

In [139]:
from datetime import datetime
from uuid import UUID, uuid4
from pydantic import BaseModel, Field


class Model(BaseModel):
    uid: UUID = Field(default_factory=uuid4)
    updated: datetime = Field(default_factory=datetime.utcnow)


m1 = Model()
m2 = Model()
print(f'{m1.uid} != {m2.uid}')
print(f'{m1.updated} != {m2.updated}')

f3e1d119-5fec-4b0b-8602-6742eb56683d != a84cf684-3839-4970-afd4-9eba66bf7be1
2022-07-12 03:26:51.684225 != 2022-07-12 03:26:51.684225


# Private model attributes

In [140]:
from datetime import datetime
from random import randint
from pydantic import BaseModel, PrivateAttr


class TimeAwareModel(BaseModel):
    _processed_at: datetime = PrivateAttr(default_factory=datetime.now)
    _secret_value: str = PrivateAttr()

    def __init__(self, **data):
        super().__init__(**data)
        # this could also be done with default_factory
        self._secret_value = randint(1, 5)


m = TimeAwareModel()
print(m._processed_at)
print(m._secret_value)

2022-07-12 11:33:15.381338
3


In [141]:
from typing import ClassVar
from pydantic import BaseModel


class Model(BaseModel):
    _class_var: ClassVar[str] = 'class var value'
    _private_attr: str = 'private attr value'

    class Config:
        underscore_attrs_are_private = True


print(Model._class_var)
print(Model._private_attr)
print(Model()._private_attr)

class var value
<member '_private_attr' of 'Model' objects>
private attr value


# Parsing data into a specified type

In [142]:
from typing import List
from pydantic import BaseModel, parse_obj_as


class Item(BaseModel):
    id: int
    name: str


item_data = [{'id': 1, 'name': 'My Item'}]
items = parse_obj_as(List[Item], item_data)
items

[Item(id=1, name='My Item')]

# Data Conversion

In [143]:
from pydantic import BaseModel


class Model(BaseModel):
    a: int
    b: float
    c: str


Model(a=3.1415, b=' 2.72 ', c=123).dict()

{'a': 3, 'b': 2.72, 'c': '123'}

# Model signature

In [144]:
import inspect
from pydantic import BaseModel, Field


class FooModel(BaseModel):
    id: int
    name: str = None
    description: str = 'Foo'
    apple: int = Field(..., alias='pear')


inspect.signature(FooModel)

<Signature (*, id: int, name: str = None, description: str = 'Foo', pear: int) -> None>

In [145]:
import inspect
from pydantic import BaseModel


class MyModel(BaseModel):
    id: int
    info: str = 'Foo'

    def __init__(self, id: int = 1, *, bar: str, **data) -> None:
        """My custom init!"""
        super().__init__(id=id, bar=bar, **data)


inspect.signature(MyModel)

<Signature (id: int = 1, *, bar: str, info: str = 'Foo') -> None>